## Wikipediaコーパスを利用した Word2vec による関係要素のembedding

In [ ]:
import glob
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import rc, font_manager
import seaborn as sns
from matplotlib import ticker
from matplotlib.ticker import ScalarFormatter

from gensim.models import Word2Vec

In [ ]:
# プロットする図のサイズ設定
plt.rcParams['figure.figsize'] = 10, 6
plt.rcParams["font.size"] = 13
# プロットする図のフォント設定
font_manager.fontManager.addfont('/Library/Fonts/ipaexg.ttf')
rc('font', family='IPAEXGothic')

sns.set_context('talk')
sns.set_style("ticks") # スタイルをticksに
sns.set(context='talk', style='ticks', font=["IPAEXGothic"], font_scale=10/6, )

- データの読み込み

In [ ]:
# data/result/に格納されている全csvファイルの読み込み
csv_files = glob.glob(os.path.join("./data/", "*.csv"))
df_list = []
for file in csv_files:
    tmp_df = pd.read_csv(file)
    df_list.append(tmp_df)
df = pd.concat(df_list, ignore_index=True)

df.shape

- 「property要素 + not (wikiPage) + not (画像)」 の関係リンクを持つデータの抽出

In [ ]:
# predの要素で, propertyが含まれる値を返す
df_prep = df[df.pred.str.contains('property')]

# 'wikiPage'を含まないprepertyを返す
df_prep = df_prep[~df_prep['pred'].str.contains('wikiPage')]

# '画像'を含まないprepertyを返す
df_prep = df_prep[~df_prep['pred'].str.contains('画像')]

df_prep.shape

- 前処理

In [ ]:
# Nan削除
df_prep = df_prep.dropna()

- objデータが数値を持つデータを省く

In [ ]:
# objデータに'数値'を含まない値を返す
df_prep = df_prep[df_prep['obj'].apply(lambda x: pd.to_numeric(x, errors='coerce')).isnull()]
        
df_prep.shape

- Word2vec のmodel 読み込み

In [ ]:
# gensim.model の読み込み
# 事前学習済みのモデルや提供されているモデルのパスを記入

# model = Word2Vec.load('../../Models/japanese-word2vec-model-builder/word2vec.gensim.model')

In [ ]:
# モデルを利用した文字のエンべディング
## 未知語に関しては, 今回はNaNで対応
def vectorize(model, word):
    try:
        output = model.wv[word]
        return output
    except:
        return np.nan

In [ ]:
# obj要素に関してはLOD同士が繋がっているため, 基本的にURIで記述されている.
# また, (県の魚:〇〇)の要素や数値データが入っている場合もあるため, それぞれ最後尾の要素を値として扱う. 
def preprocessing(obj):
    if type(obj) != str:
        output = obj
    else:
        output = obj.split("/")[-1]
    return output

- word2vec でのエンべディングを実施する
    - key, obj から pred(rel) の分散表現ベクトルを求める

In [ ]:
# 手法
## key は, 基本的flagsの形式である. exe: 沖縄県, ボブサップ, さつまいも
## obj は, URI形式と(数)単語を含む自然文の形式である. exe: http://ja.dbpedia.org/resource/〇〇, 那覇市
    
cp_df = df_prep.copy()
cp_df.shape

In [ ]:
## key のvectorization
### key に関しては,preprocessingは特に必要ないと仮定

cp_df['key_vec'] = cp_df['key'].map(lambda key:vectorize(model, key))
is_key_notnull = cp_df['key_vec'].notnull()
print("登録済みkeyの要素数は:",len(cp_df[is_key_notnull]))
print("登録済みkeyのユニーク要素の個数は:", \
      cp_df[is_key_notnull]['key'].nunique())

In [ ]:
## obj のpreprocessing
  ## DBpedia上で登録されているuri情報を取り除く

uri = 'http://ja.dbpedia.org/resource/'
obj_list = cp_df['obj'].map(lambda obj:obj.replace(uri,''))
obj_list = obj_list.map(lambda obj:obj.replace('※',''))
obj_list = obj_list.str.replace('[()]','', regex=True)
obj_list = obj_list.str.replace('[（）]',' ', regex=True)
obj_list = obj_list.str.replace('.+[「『]|[」』]','', regex=True)
obj_list = obj_list.str.strip()

In [ ]:
def obj_preprocessing_1(obj):
    if type(obj) != str:
        output = obj
    elif len(obj.split("：")) != 1:
        output = obj.split("：")[1]
    elif len(obj.split(":")) != 1:
        output = obj.split(":")[1]
    else:
        output = obj
    return output.strip()  
        
def obj_preprocessing_2(obj):
    if type(obj) != str:
        output = obj
    elif len(obj.split("_")) != 1:
        output = obj.split("_")[0]
    elif len(obj.split("、")) != 1:
        output = obj.split("、")[0]
    elif len(obj.split(" ")) != 1:
        output = obj.split(" ")[0]
    elif len(obj.split("、")) != 1:
        output = obj.split("、")[0]
    elif len(obj.split("・")) != 1:
        output = obj.split("・")[0]
    elif len(obj.split("・")) != 1:
        output = obj.split("・")[0]
    else:
        output = obj
    return output.strip()

In [ ]:
obj_list = obj_list.map(lambda obj:obj_preprocessing_1(obj))
obj_list = obj_list.map(lambda obj:obj_preprocessing_2(obj))
cp_df['obj_vec'] = obj_list.map(lambda obj:vectorize(model, obj))
cp_df['pre_obj'] = obj_list
is_obj_notnull = cp_df['obj_vec'].notnull()
print("登録済みobjの要素数は:",len(cp_df[is_obj_notnull]))
print("登録済みobjのユニーク要素の個数は:", \
      cp_df[is_obj_notnull]['obj'].nunique())

- key + rel = obj が成り立つと仮定する.
  - そこで, rel は, obj - key で求められるとする.
  - その後, 求めたrel を要素ごとの値ではなく関係ごとの値として定める.

In [ ]:
cp_df = cp_df.loc[is_key_notnull & is_obj_notnull]

cp_df['pred_vec'] = cp_df['obj_vec'] - cp_df['key_vec']
# display(cp_df[is_obj_notnull].head(3))

print("pred_vec がnullの要素数は：",cp_df['pred_vec'].isnull().sum())

In [ ]:
## 合成ベクトル or 重心ベクトルを求めてみる.
sum_vec = cp_df.groupby('pred')['pred_vec'].apply(lambda x:np.sum(x))
cent_vec = cp_df.groupby('pred')['pred_vec'].apply(lambda x:np.sum(x)/x.count())
cp_pred_df = pd.DataFrame([sum_vec,cent_vec]).T

cp_pred_df.columns = ['sum_vec','cent_vec']
# display(cp_pred_df.sample(2))

- wikipediaをコーパスとする分散表現の獲得が達成
    - cp1_pred_df ← pred(rel)が持つURI形式を外し,単語にしたものをembedding
    - cp_pred_df ← key, obj をembeddingし, 仮定式： key - rel = obj を置いたときの合成ベクトルと重心ベクトル

- 階層クラスタリングを用いて曖昧さを考慮したまとまりの作成を実施する

In [ ]:
from scipy.cluster.hierarchy import linkage, fcluster, cophenet, dendrogram

- 要素毎(predでまとめない)の階層クラスタリング

In [ ]:
# cp_df.duplicated(subset=['key','obj']).sum()
# 179

is_key_obj_duplicated = cp_df.duplicated(subset=['key','pre_obj'])
cp_element_df = cp_df.loc[~is_key_obj_duplicated,['key','pred','pre_obj','pred_vec']].reset_index(drop=True)
cp_element_df['label'] = cp_element_df['key'] + '/' + cp_element_df['pre_obj']

In [ ]:
cp_element_df.loc[:,['key','pred','pre_obj']].nunique()

In [ ]:
cp_element_df.shape

- 知っていたが, 流石に1008は多すぎるな.

In [ ]:
list_pred_vec = cp_element_df['pred_vec'].tolist()
pred_vec_idnex = cp_element_df.index

result = linkage(list_pred_vec,
                 method='ward',
                 metric='euclidean')

In [ ]:
threshold = 0.1 * np.max(result[:, 2])
threshold2 = 0.15 * np.max(result[:, 2])
threshold3 = 0.2 * np.max(result[:, 2])
threshold4 = 0.25 * np.max(result[:, 2])

In [ ]:
fig, ax = plt.subplots(figsize = (15,10))

dendrogram(result,
           labels=list(cp_element_df['label']),
           color_threshold=threshold3)

ax.axhline(threshold3, linestyle='--', color='r')
sns.despine()
# plt.title("pred_cluster_02")
# ax.legend()
ax.set(xlabel = '', ylabel='Threshold')
plt.xticks(fontsize=13)

ax.yaxis.set_major_formatter(ScalarFormatter(useMathText=True)) 
# plt.savefig('./output/02_cluster.png', transparent = True, bbox_inches='tight')  

In [ ]:
# criterion は, fcluster 作成でのクラスタ選びのアルゴリズム
# 他にもいろんなアルゴリズムがある
# fcluster の出力_array のindex は, 入力データのindex に属する 

cluster1 = fcluster(result,
                    threshold,
                    criterion='distance')

cluster2 = fcluster(result,
                    threshold2,
                    criterion='distance')

cluster3 = fcluster(result,
                    threshold3,
                    criterion='distance')

cluster4 = fcluster(result,
                    threshold4,
                    criterion='distance')

In [ ]:
# 階層クラスター分析の結果をDataFrame化
_cluster = pd.DataFrame({'class_thre_0.1':cluster1,
                         'class_thre_0.15':cluster2,
                         'class_thre_0.2':cluster3,
                         'class_thre_0.25':cluster4,
                        }
                        , index = pred_vec_idnex)

display(_cluster.head())

In [ ]:
# 元データと分析結果を結合
cluster_df = pd.concat([cp_element_df, _cluster] ,axis=1)
display(cluster_df.head())

In [ ]:
cluster_df.shape

In [ ]:
pd.to_pickle(cluster_df, '../../pickles/cluster_df.pkl')